In [1]:
import selenium
import pandas as pd
from bs4 import BeautifulSoup
import time
import re
from selenium import webdriver  
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [2]:
from selenium.webdriver import Chrome, Firefox

In [3]:
#browser=Chrome()
browser=Firefox()

In [4]:
url = "https://www.commonsensemedia.org/game-reviews"
#url="https://www.commonsensemedia.org/#content"
browser.get(url)

In [5]:
from selenium.webdriver.support.ui import Select

In [6]:
#Handeling cookies dialougue box
time.sleep(2)

try:
    closeCookieButton = browser.find_element_by_id('onetrust-pc-btn-handler').click()
    Confirmmychoices = browser.find_element_by_class_name('onetrust-close-btn-handler').click()
    #closeCookieButton.click()
except:
    print('No popup dialog to close...continue..')

In [7]:
#Scrapping the review page common for both Parents and Kids
def scrap_reviews():
    review = ""
    while True:
        try:
            element = WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.LINK_TEXT, "continue reading"))) 
            links = browser.find_elements_by_partial_link_text('continue reading')
            #links= browser.find_elements_by_class_name("read-more-close")
            if len(links) != 0:
                for link in links:
                    link.click()
                    time.sleep(1)
        except:
            print("")
        html=browser.page_source
        soup=BeautifulSoup(html,'html.parser')
        user_reviews=soup.find_all('div', {'class':'review-wrapper'})
        for user_review in user_reviews:
            review += user_review.text
        try:
            next_review_page = WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.LINK_TEXT, "next ›"))) 
            #next_review_page = browser.find_elements_by_partial_link_text('next ›')
            next_review_page.click()
            time.sleep(1)
        except:
            break
    return review

In [8]:
#Function will navigate to parents review page and will collect all parents review 
def scrap_parent_reviews():
    parent_reviews=None
    parent_rating=None
    try:
        parent_review_element = WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/div[1]/div[1]/div[3]/div/div[1]/div/div/div[3]/a')))
        parent_rating_HTML = browser.find_element_by_xpath("//div[@class='user-review-statistics adult']").get_attribute('innerHTML')
        parent_rating = re.findall(r'rating-\d{1}',parent_rating_HTML)
        parent_rating =  re.findall(r'\d{1}',parent_rating[0])[0]
    except:
        return parent_reviews, parent_rating
    no_of_retry=0
    while no_of_retry != 3:
        try:
            parent_review_element.click()
            time.sleep(2)
            break
        except:
            no_of_retry += 1
            if no_of_retry == 3:
                parent_reviews='review present but not collected'
                break
            time.sleep(1)
    if no_of_retry < 3:
        parent_reviews=scrap_reviews()
        
    return parent_reviews, parent_rating 

In [9]:
#Function will navigate to Kids review page and will collect all parents review 

def scrap_kids_reviews():
    kids_rating = None
    kids_reviews = None
    try:
        kids_review_element = WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/div[1]/div[1]/div[3]/div/div[2]/div/div/div[3]/a')))
        kids_rating_HTML = browser.find_element_by_xpath("//div[@class='user-review-statistics child']").get_attribute('innerHTML')
        kids_rating = re.findall(r'rating-\d{1}',kids_rating_HTML)
        kids_rating =  re.findall(r'\d{1}',kids_rating[0])[0]
    except:
        return kids_reviews, kids_rating    
    no_of_retry=0
    while no_of_retry != 3:
        try:
            kids_review_element.click()
            time.sleep(2)
            break
        except:
            no_of_retry += 1
            if no_of_retry == 3:
                kids_reviews='review present but not collected'
                break
            time.sleep(1)
    if no_of_retry < 3:
        kids_reviews=scrap_reviews()
    return kids_reviews, kids_rating

In [10]:
#This function will collect all the required data of the game
def scrap_game_page(link):
    no_of_retry=0
    while no_of_retry != 3:
        try:
            game_review_link=browser.find_element_by_partial_link_text(link.text.strip())
            game_review_link.click()
            break
        except:
            no_of_retry += 1
            time.sleep(.5)
    if no_of_retry == 3:
        return None       
    time.sleep(3)
    html=browser.page_source
    soup=BeautifulSoup(html,'html.parser')
    game_desc_url = browser.current_url
    # Game Title
    try:
        game_title=soup.find('h1').text.strip()
    except:
        game_title= None
    try:
        age_data=soup.find('div',{'class':'csm-green-age'}).text.split(" ")[1].strip()
    except:
        age_data=None
    try:    
        parent_need_to_know=soup.find('div',{'class':'shutter-summary-pane panel-pane pane-entity-field pane-node-field-parents-need-to-know with-foot'}).text.strip()
    except:
        parent_need_to_know = None
    try:
        game_description=story=soup.find('div',{'class':'field field-name-field-what-is-story field-type-text-long field-label-hidden'}).text.strip()
    except:
        game_description = None
    try:
        talk_kid_about_this=soup.find('div',{'class':'field field-name-field-family-topics field-type-text-long field-label-hidden'}).text.strip() 
    except:
        talk_kid_about_this = None    
    product_details=soup.find_all('ul',{'id':'review-product-details-list'})
    try:
        game_theme_info=product_details[0].find('li',{'class': 'themes'})
        game_theme = game_theme_info.text.split(":")[1].strip()
    except:
        game_theme = None
    try:
        game_developer_info=product_details[0].find('li',{'class': 'publishers'})
        game_developer=game_developer_info.text.split(":")[1].strip()
    except:
        game_developer=None
    try:
        game_title=soup.find('h1').text.strip()
    except:
        game_title= None
        
    try:
        product_details=browser.find_element_by_id('review-product-details-list').get_attribute('innerHTML')
        a = re.findall(r'genre/.*',product_details)
        b = re.findall(r'>.*</a',a[0])
        genre = b[0].replace(">","").replace("</a","")
    except:
        genre=None
    parent_reviews, parent_ratings = scrap_parent_reviews()
    browser.get(game_desc_url)
    time.sleep(2)
    kids_reviews, kids_rating = scrap_kids_reviews()
    browser.get("https://www.commonsensemedia.org/game-reviews?page={}".format(page_no))
    time.sleep(2)
    return [game_title, age_data, game_description, parent_need_to_know, talk_kid_about_this, game_theme, game_developer, 
            genre, parent_reviews,parent_ratings, kids_reviews, kids_rating]
    

In [11]:
#Main function where script will hover over all the pages, collect required data of game
scrapped_data=[]
page_no=0
while True:
    html=browser.page_source
    soup=BeautifulSoup(html,'html.parser')
    links_selected=soup.find_all('strong',{'class':'field-content'})
    for link in links_selected:
        game_scrapped=scrap_game_page(link)
        if game_scrapped is not None:
            scrapped_data.append(game_scrapped)
            #print(scrapped_data)
    try:
        next_page_button = browser.find_elements_by_partial_link_text('next ›')
        next_page_button[0].click()
        time.sleep(1)
        print('Moving to next page # ',page_no)
        page_no += 1
    except:
        break

KeyboardInterrupt: 

In [12]:
#Creating Dataframe of scrapped data
df = pd.DataFrame(scrapped_data, columns = ['Title', 'Age', 'Parents need to know', 'Game Description',
                                            'Kids say about this game', 'Game theme', 'Developer','Genre', 
                                            'Parents Reviews','Parents Rating', 'Kids Reviews', 'kids_rating'])

In [14]:
pd.set_option('display.max_colwidth', 50)
df

,Title,Age,Parents need to know,Game Description,Kids say about this game,Game theme,Developer,Genre,Parents Reviews,Parents Rating,Kids Reviews,kids_rating
0,Hyrule Warriors: Age of Calamity,13+,Return once again to the world of Hyrule in HY...,What parents need to know\n\n\nParents need to...,Families can talk about violence in video game...,"Magic and Fantasy, Princesses, Fairies, Mermai...",Nintendo of America,Action/Adventure,\nBloodless violence in well made prequel to B...,5,\nFun and Slightly Challenging \nOverall this...,5
1,Pac-Man Mega Tunnel Battle,8+,PAC-MAN MEGA TUNNEL BATTLE is the latest title...,What parents need to know\n\n\nParents need to...,Families can talk about nostalgia. Does Pac-Ma...,None,Bandai Namco,Arcade,None,None,None,None
2,Kingdom Hearts: Melody of Memory,10+,Return once more to the world of Kingdom Heart...,What parents need to know\n\n\nParents need to...,Families can talk about music and music apprec...,"Magic and Fantasy, Adventures, Friendship, Mon...",Square Enix,Music &amp; Dance,None,None,None,None
3,The Falconeer,10+,THE FALCONEER is an epic fantasy tale in which...,What parents need to know\n\n\nParents need to...,Families can talk about violence in video game...,"Adventures, Wild Animals",Wired Productions,Action/Adventure,\n linguistical game\nThe game has a language ...,2,None,None
4,Sackboy: A Big Adventure,8+,"In SACKBOY: A BIG ADVENTURE, our titular hero ...",What parents need to know\n\n\nParents need to...,Families can talk about violence in video game...,Adventures,Sony Interactive Entertainment,Action/Adventure,\nNot the best of the Little Big Planet series...,5,None,None
5,Need for Speed: Hot Pursuit Remastered,10+,In NEED FOR SPEED: HOT PURSUIT REMASTERED -- a...,What parents need to know\n\n\nParents need to...,Families can talk about road safety. In Need f...,Cars and Trucks,Electronic Arts,Racing,None,None,None,None
6,Call of Duty: Black Ops Cold War,18+,The story-driven campaign of CALL OF DUTY: BLA...,What parents need to know\n\n\nParents need to...,Families can talk about violence in video game...,Adventures,Activision,First-Person Shooter,\nJust as mature as the other games in this se...,5,\nAmazing game can turn off gore highly recomm...,5
7,Game & Watch: Super Mario Bros.,8+,GAME & WATCH: SUPER MARIO BROS. is the latest ...,What parents need to know\n\n\nParents need to...,Families can talk about nostalgia. What's the ...,"Adventures, Monsters, Ghosts, and Vampires",Nintendo of America,Action/Adventure,\nIt's rated for everyone\nRated for everyone ...,5,None,None
8,Astro's Playroom,6+,"In ASTRO'S PLAYROOM, players guide Astro throu...",What parents need to know\n\n\nParents need to...,Families can talk about progress. How have som...,"Adventures, Robots",Sony Interactive Entertainment,Arcade,\nCute but short platformer has mild cartoon v...,4,None,None
9,Just Dance 2021,10+,Celebrating ten years of the Just Dance series...,What parents need to know\n\n\nParents need to...,Families can talk about using games as exercis...,"Friendship, Music and Sing-Along",UbiSoft,Music &amp; Dance,"\nMuch like previous games, just a few songs w...",3,,1


In [ ]:
#Storing dataframe data to CSV file
save_csv = df.to_csv('webminingProject.csv')